In [1]:
from common import *

DAY = 10
lines = get_input_lines(DAY)

In [2]:
lines = '''.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...'''.splitlines()

In [3]:
from structures import Map

directions = {
    (0, -1): '-J7S',
    (0, 1): '-LFS',
    (-1, 0): '|LJS',
    (1, 0): '|7FS',
}
directions_inv = {(k[0] * -1, k[1] * -1): v for k, v in directions.items()}

def is_connected(y: int, x: int, value: int, new_y: int, new_x: int, new_value: int) -> bool:
    diff = (new_y - y, new_x - x)
    return lines[y][x] in directions[diff] and lines[new_y][new_x] in directions_inv[diff]

In [4]:
dmap = Map(len(lines), len(lines[0]))
y, x = Map.from_object(lines).find_value('S')

dmap.map[y][x] = 0
while dmap.calc_inc_step(condition=is_connected):
    pass

dmap.show()

 . 59 60 61 62 63 64 69 68 61 60 51 50 43 42 41  .  .  .  . 
 . 58 47 46 45 44 65 70 67 62 59 52 49 44 39 40  .  .  .  . 
 . 57 48  . 42 43 66 69 66 63 58 53 48 45 38 37  .  .  .  . 
55 56 49 50 41 40 67 68 65 64 57 54 47 46  . 36 35 34  .  . 
54 53 52 51  . 39 38  .  .  . 56 55  0  1 20 21 22 33 32  . 
 .  .  .  . 35 36 37  .  . 14 13  2  1  2 19 18 23 24 31 30 
 .  .  .  . 34 33  . 19 18 15 12  3  4  3  . 17 16 25 26 29 
 .  .  .  .  . 32 21 20 17 16 11  6  5  4  9 10 15  . 27 28 
 .  .  .  . 30 31 22 23 24  . 10  7  .  5  8 11 14  .  .  . 
 .  .  .  . 29 28 27 26 25  .  9  8  .  6  7 12 13  .  .  . 


In [5]:
result = max(max(line) for line in dmap.map)
# assert result == 6867

In [6]:
def change_coord(x):
    return x * 2 + 2

pmap = Map(change_coord(dmap.h), change_coord(dmap.w))

max_d = max(max(line) for line in dmap.map)
start_y, start_x = dmap.find_value(max_d)
pmap.map[change_coord(start_y)][change_coord(start_x)] = max_d

for y, x, _ in dmap.get_nearest_points_by_value(start_y, start_x, max_d - 1):
    pmap.map[change_coord(start_y) + y - start_y][change_coord(start_x) + x - start_x] = dmap.map[y][x]
    pmap.map[change_coord(y)][change_coord(x)] = dmap.map[y][x]
    while True:
        num = dmap.map[y][x]
        for new_y, new_x, new_num in dmap.get_nearest_points_by_value(y, x, num - 1):
            pmap.map[change_coord(new_y)][change_coord(new_x)] = new_num
            pmap.map[change_coord(y) + new_y - y][change_coord(x) + new_x - x] = new_num
            y, x = new_y, new_x
            break
        else:
            break

pmap.show()

 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  . 59 59 60 60 61 61 62 62 63 63 64  . 69 68 68  . 61 60 60  . 51 50 50  . 43 42 42 41 41  .  .  .  .  .  .  .  .  . 
 .  .  .  . 58  .  .  .  .  .  .  .  .  . 64  . 69  . 67  . 61  . 59  . 51  . 49  . 43  .  .  . 40  .  .  .  .  .  .  .  .  . 
 .  .  .  . 58  . 47 46 46 45 45 44 44  . 65  . 70  . 67  . 62  . 59  . 52  . 49  . 44  . 39 39 40  .  .  .  .  .  .  .  .  . 
 .  .  .  . 57  . 47  .  .  .  .  . 43  . 65  . 69  . 66  . 62  . 58  . 52  . 48  . 44  . 38  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  . 57  . 48  .  .  . 42 42 43  . 66  . 69  . 66  . 63  . 58  . 53  . 48  . 45  . 38 37 37  .  .  .  .  .  .  .  .  . 
 .  .  .  . 56  . 48  .  .  . 41  .  .  . 66  . 68  . 65  . 63  . 57  . 53  . 47  . 45  .  .  . 36  .  .  .  . 

In [7]:
pmap.map[0][0] = 'O'

while pmap.calc_step(
    point_filter=lambda y, x, value: value == 'O',
    target_filter=lambda y, x, value, new_y, new_x, new_value: new_value == -1,
    target_value='O',
):
    pass

pmap.show()

 O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O 
 O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O 
 O  O  O  O 59 59 60 60 61 61 62 62 63 63 64  O 69 68 68  O 61 60 60  O 51 50 50  O 43 42 42 41 41  O  O  O  O  O  O  O  O  O 
 O  O  O  O 58  .  .  .  .  .  .  .  .  . 64  O 69  . 67  O 61  . 59  O 51  . 49  O 43  .  .  . 40  O  O  O  O  O  O  O  O  O 
 O  O  O  O 58  . 47 46 46 45 45 44 44  . 65  O 70  . 67  O 62  . 59  O 52  . 49  O 44  . 39 39 40  O  O  O  O  O  O  O  O  O 
 O  O  O  O 57  . 47  O  O  O  O  O 43  . 65  O 69  . 66  O 62  . 58  O 52  . 48  O 44  . 38  O  O  O  O  O  O  O  O  O  O  O 
 O  O  O  O 57  . 48  O  O  O 42 42 43  . 66  O 69  . 66  O 63  . 58  O 53  . 48  O 45  . 38 37 37  O  O  O  O  O  O  O  O  O 
 O  O  O  O 56  . 48  O  O  O 41  .  .  . 66  O 68  . 65  O 63  . 57  O 53  . 47  O 45  .  .  . 36  O  O  O  O 

In [8]:
result = 0
for y, line in enumerate(pmap.map):
    for x, value in enumerate(line):
        if value == -1 and x % 2 == 0 and y % 2 == 0:
            result += 1
# assert result == 595

In [9]:
# alternative solution

dmap = Map(len(lines), len(lines[0]))
y, x = Map.from_object(lines).find_value('S')

dmap.map[y][x] = 0
while dmap.calc_inc_step(condition=is_connected):
    pass

dmap.show()

 . 59 60 61 62 63 64 69 68 61 60 51 50 43 42 41  .  .  .  . 
 . 58 47 46 45 44 65 70 67 62 59 52 49 44 39 40  .  .  .  . 
 . 57 48  . 42 43 66 69 66 63 58 53 48 45 38 37  .  .  .  . 
55 56 49 50 41 40 67 68 65 64 57 54 47 46  . 36 35 34  .  . 
54 53 52 51  . 39 38  .  .  . 56 55  0  1 20 21 22 33 32  . 
 .  .  .  . 35 36 37  .  . 14 13  2  1  2 19 18 23 24 31 30 
 .  .  .  . 34 33  . 19 18 15 12  3  4  3  . 17 16 25 26 29 
 .  .  .  .  . 32 21 20 17 16 11  6  5  4  9 10 15  . 27 28 
 .  .  .  . 30 31 22 23 24  . 10  7  .  5  8 11 14  .  .  . 
 .  .  .  . 29 28 27 26 25  .  9  8  .  6  7 12 13  .  .  . 


In [10]:
processed = set()
y, x = dmap.find_value(0)
rev = False
while True:
    next_value = dmap.map[y][x] + 1 if not rev else dmap.map[y][x] - 1
    if next_value < 0:
        break
    nearest = list(dmap.get_nearest_points_by_value(y, x, next_value))
    if not nearest and not rev:
        rev = True
        continue
    if not nearest:
        break
    next_y, next_x, _ = nearest[0]
    processed.add((next_y, next_x))
    y_diff, x_diff = next_y - y, next_x - x
    colors_diffs_by_diff = {
        (0, -1): [(1, 0), (-1, 0)],
        (0, 1): [(-1, 0), (1, 0)],
        (-1, 0): [(0, -1), (0, 1)],
        (1, 0): [(0, 1), (0, -1)],
    }
    in_diff, out_diff = colors_diffs_by_diff[(y_diff, x_diff)]
    for some_diff, diff_char in ((in_diff, 'I'), (out_diff, 'O')):
        for y_color, x_color in ((next_y, next_x), (y, x)):
            while True:
                y_color += some_diff[0]
                x_color += some_diff[1]
                if not (0 <= y_color < dmap.h) or not (0 <= x_color < dmap.w):
                    break
                color_value = dmap.map[y_color][x_color]
                if color_value not in (-1, 'I', 'O'):
                    break
                dmap.map[y_color][x_color] = diff_char
    y, x = next_y, next_x

dmap.show()

 I 59 60 61 62 63 64 69 68 61 60 51 50 43 42 41  I  I  I  I 
 I 58 47 46 45 44 65 70 67 62 59 52 49 44 39 40  I  I  I  I 
 I 57 48  I 42 43 66 69 66 63 58 53 48 45 38 37  I  I  I  I 
55 56 49 50 41 40 67 68 65 64 57 54 47 46  O 36 35 34  I  I 
54 53 52 51  I 39 38  O  O  O 56 55  0  1 20 21 22 33 32  I 
 I  I  I  I 35 36 37  O  O 14 13  2  1  2 19 18 23 24 31 30 
 I  I  I  I 34 33  O 19 18 15 12  3  4  3  O 17 16 25 26 29 
 I  I  I  I  I 32 21 20 17 16 11  6  5  4  9 10 15  I 27 28 
 I  I  I  I 30 31 22 23 24  I 10  7  I  5  8 11 14  I  I  I 
 I  I  I  I 29 28 27 26 25  I  9  8  I  6  7 12 13  I  I  I 


In [11]:
counts = {}
for char in 'IO':
    counts[char] = sum(sum(1 for v in line if v == char) for line in dmap.map)
min(counts.values())

8